In [80]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
from datetime import datetime
import numpy as np
import numpy.random as random
import scipy as sp
from scipy import stats
from scipy import integrate
from pandas import Series, DataFrame
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
import matplotlib as mlp
import matplotlib.pyplot as plt
import japanize_matplotlib
import math
from random import Random

%matplotlib inline
%matplotlib ipympl

In [81]:
def agg_data_for_unit():
    data_sets = pd.DataFrame()
    beacon_files = os.listdir('raw_data/')
    pos_files = os.listdir('POS_RAW_data/')

    beacon_files.sort()
    pos_files.sort()
    
    if beacon_files[0] == ".DS_Store":
        beacon_files.pop(0)

    if pos_files[0] == ".DS_Store":
        pos_files.pop(0)

    pos_files = list(map(lambda x: x[:14], pos_files))[::2]

    for beacon_file, pos_file in zip(beacon_files, pos_files):
        date_beacon = beacon_file[9:17]
        date_pos    = pos_file[6:14]

        if date_beacon != date_pos:
            continue
        else:
            date = date_beacon
        
        t_start = datetime.strptime(f'{date} 09:00', '%Y%m%d %H:%M').timestamp()
        t_end   = datetime.strptime(f'{date} 21:00', '%Y%m%d %H:%M').timestamp()
        rssi    = 70
        beacon_data = pd.read_csv(f"raw_data/{beacon_file}")
        pos_data1 = pd.read_csv(f"POS_RAW_data/{pos_file}_01.csv")
        pos_data2 = pd.read_csv(f"POS_RAW_data/{pos_file}_02.csv")

        pos_data1 = pos_data1[["商品コード", "商品名称（または券名称）", "単価", "数量", "合計金額"]]
        pos_data2 = pos_data2[["商品コード", "商品名称（または券名称）", "単価", "数量", "合計金額"]]
        pos_data  = pd.concat([pos_data1, pos_data2])

        pos_data[["単価", "数量", "合計金額"]] = pos_data[["単価", "数量", "合計金額"]].map(lambda x: int(x))
        pos_data = pos_data.groupby(["商品コード", "商品名称（または券名称）"]).sum()

        pos_data["単価"] = pos_data["合計金額"] / pos_data["数量"]
        pos_data["単価"] = pos_data["単価"].astype(int)

        beacon_data.columns = ["No.", "mac-address", "distance", "rssi", "random", "timestamp"]
        beacon_data = beacon_data[beacon_data["random"] == 1]
        beacon_data = beacon_data[beacon_data["timestamp"] >= t_start]
        beacon_data = beacon_data[beacon_data["timestamp"] <= t_end]
        beacon_data = beacon_data[beacon_data["rssi"] < rssi]
        beacon_data = beacon_data.drop_duplicates("mac-address")

        per_unit = pd.DataFrame(
            {
                "date":[date],
                "総ビーコン数": [len(beacon_data)],
                "総売上点数": [pos_data["数量"].sum()],
                "総売上": [pos_data["合計金額"].sum()],
            }
        )
        data_sets = pd.concat([data_sets, per_unit])

    return data_sets.reset_index(drop=True)

In [82]:
def poisson_distribution(lambda_, k):
    return stats.poisson.pmf(round(k), lambda_)

def normal_distribution(x, mu, sigma):
    return 1 / np.sqrt(2 * np.pi * (sigma ** 2)) * np.exp(-((x - mu) ** 2) / (2 * (sigma ** 2)))

def uniform_distribution(x, alpha, beta):
    if alpha > beta:
        alpha, beta = beta, alpha

    if (alpha <= x) and (x <= beta):
        return 1 / (beta - alpha)
    else:
        return 0.0

In [83]:
def Leap_Flog(h_dash, θ, p, ε, args=(), max_iterate=1e3):
    ite = 0
    while ite < max_iterate:
        tmp = -1 / 2 * h_dash(θ, *args)
        p = p + ε * tmp
        θ = θ + ε * p

        tmp = -1 / 2 * h_dash(θ, *args)
        p = p + ε * tmp

        ite = ite + 1
    
    return θ, p

In [84]:
def Leap_Flog_Cus(h_dash, θ, p, ε, args=(), max_iterate=1e3):
    ite = 0
    α = 0.9
    while ite < max_iterate:
        tmp = -1 / 2 * h_dash(θ, *args)
        p = (α * p + (1 - α) * ε * tmp) / α
        θ = θ + ε * p

        tmp = -1 / 2 * h_dash(θ, *args)
        p = (α * p + (1 - α) * ε * tmp) / α

        ite = ite + 1
    
    return θ, p

In [85]:
class PoissonRegression_On_Bayes:
    def __init__(self, isGLMM=False, hamiltonian_ε=0.0001, hamiltonian_ite=100, gauss_quadrature_dim=3200, random_state=None):
        self.alpha   = np.array([], dtype=np.float64)
        self.alpha0  = np.float64(0.0)
        self.isGLMM  = isGLMM
        self.gauss_s = np.float64(0.1)
        self.max_dim = gauss_quadrature_dim
        self.hamiltonian_ε = hamiltonian_ε
        self.hamiltonian_ite = hamiltonian_ite
        self.sampling_alpha   = np.array([], dtype=np.float64)
        self.sampling_alpha0  = np.array([], dtype=np.float64)
        self.sampling_gauss_s = np.array([], dtype=np.float64)

        if random_state != None:
            self.random = np.random
            self.random.seed(seed=random_state)
        else:
            self.random = np.random

    def sampling(self, x_train, y_train, iter_num=1e6, burnin=1e3):
        if (x_train.ndim != 2) or (y_train.ndim != 1):
            print(f"x_train dims = {x_train.ndim}")
            print(f"y_train dims = {y_train.ndim}")
            print("エラー：：次元数が一致しません。")
            return False

        if type(x_train) is pd.core.frame.DataFrame:
            x_train = x_train.to_numpy()

        if type(y_train) is pd.core.series.Series:
            y_train = y_train.to_numpy()
        
        num, s = x_train.shape
        y_train = y_train.reshape([num, 1])

        #正規方程式
        A = np.hstack([x_train, np.ones([num, 1])])
        b = np.log(y_train).reshape([num])

        x = np.dot(np.linalg.inv(np.dot(A.T, A)), np.dot(A.T, b))
        self.alpha, self.alpha0 = x[0:s].reshape([1, s]), x[s]
        self.gauss_s            = np.float64(0.1)

        def likelihood_GLMM_diff(x, speci_dim):
            alpha, alpha0, gauss_s = x[:-2], x[-2], x[-1]

            exp_index   = (np.sum(alpha * x_train, axis=1) + alpha0).reshape([num, 1])
            upper_lim   = exp_index < 650
            upper_rev   = upper_lim == False
            exp_index_T = exp_index * upper_lim
            exp_index_F = 650       * upper_rev
            
            lambda_vec = np.exp(exp_index_T)
            lambda_vec = lambda_vec.reshape([num, 1])

            def vectorlize_quad(lambda_, y_train_indi, gauss_s, speci_dim):
                def calc_likelihood(r, λ, k, s):
                    λ_r  = λ * np.exp(r)
                    poisson = poisson_distribution(λ_r, k)
                    normal  = normal_distribution(r, 0, s)
                    return poisson * normal

                def calc_λ(r, λ, k, s):
                    λ_r  = λ * np.exp(r)
                    poisson = poisson_distribution(λ_r, k)
                    normal  = normal_distribution(r, 0, s)
                    return poisson * normal * λ

                def calc_squared_r(r, λ, k, s):
                    λ_r  = λ * np.exp(r)
                    poisson = poisson_distribution(λ_r, k)
                    normal  = normal_distribution(r, 0, s)
                    return poisson * normal * (r ** 2)

                range_σ    = max(10 * gauss_s, 1)
                likelihood = integrate.fixed_quad(calc_likelihood, -range_σ, range_σ, args=(lambda_, y_train_indi, gauss_s), n=speci_dim)[0]
                diff_alpha = integrate.fixed_quad(calc_λ,          -range_σ, range_σ, args=(lambda_, y_train_indi, gauss_s), n=speci_dim)[0]
                diff_s     = integrate.fixed_quad(calc_squared_r,  -range_σ, range_σ, args=(lambda_, y_train_indi, gauss_s), n=speci_dim)[0]
                
                tmp = np.zeros(2)
                tmp[0]   = diff_alpha / (likelihood + 1e-16)
                tmp[1]   = diff_s     / (likelihood + 1e-16)
                return tmp[0], tmp[1]

            diff_calc         = np.frompyfunc(vectorlize_quad, 4, 2)(lambda_vec, y_train, gauss_s, speci_dim)
            diff_alpha_calc   = diff_calc[0].astype(float).reshape([num, 1])
            diff_gauss_s_calc = diff_calc[1].astype(float).reshape([num, 1])

            diff_alpha_calc   = y_train - diff_alpha_calc
            diff_gauss_s_calc = (diff_gauss_s_calc / (self.gauss_s ** 2) - 1) / self.gauss_s

            diff_alpha_calc   = diff_alpha_calc   * upper_lim + (y_train - np.exp(exp_index_F).reshape([num, 1])) * upper_rev
            diff_gauss_s_calc = diff_gauss_s_calc * upper_lim

            diff_alpha   = np.sum(diff_alpha_calc * x_train, axis=0).reshape([1, s]) / num
            diff_alpha0  = np.sum(diff_alpha_calc)                                   / num
            diff_gauss_s = np.sum(diff_gauss_s_calc)                                 / num

            return -np.hstack([diff_alpha.reshape(diff_alpha.shape[1]), diff_alpha0, diff_gauss_s])
        
        def vectorlize_quad(lambda_, y_train_indi, gauss_s, speci_dim):
            def calc_likelihood(r, λ, k, s):
                λ_r  = λ * np.exp(r)
                poisson = poisson_distribution(λ_r, k)
                normal  = normal_distribution(r, 0, s)
                return poisson * normal

            range_σ = max(10 * gauss_s, 1)
            tmp = integrate.fixed_quad(calc_likelihood, -range_σ, range_σ, args=(lambda_, y_train_indi, gauss_s), n=speci_dim)[0]
            tmp = np.log(tmp + 1e-16)
            return tmp
        
        def likelihood_GLM_diff(x):
            alpha, alpha0 = x[:-1], x[-1]
            lambda_vec = np.exp(np.sum(alpha * x_train, axis=1) + alpha0)
            lambda_vec = lambda_vec.reshape([num, 1])

            diff_alpha_calc   = y_train - lambda_vec

            diff_alpha   = np.sum(diff_alpha_calc * x_train, axis=0).reshape([1, s]) / num
            diff_alpha0  = np.sum(diff_alpha_calc)                                   / num

            return -np.hstack([diff_alpha.reshape(diff_alpha.shape[1]), diff_alpha0])
        
        def vectorlize_non_quad(lambda_, y_train_indi):
            tmp = poisson_distribution(lambda_, y_train_indi)
            tmp = np.log(tmp + 1e-16)
            return tmp
        
        py_vectorlize_quad     = np.frompyfunc(vectorlize_quad, 4, 1)
        py_vectorlize_non_quad = np.frompyfunc(vectorlize_non_quad, 2, 1)

        now_ite  = 0
        while now_ite < iter_num:
            tmp_alpha, tmp_alpha0 = self.alpha, self.alpha0
            tmp_gauss_s           = self.gauss_s

            if self.isGLMM == True:
                x = np.hstack([tmp_alpha.reshape(tmp_alpha.shape[1]), tmp_alpha0, tmp_gauss_s])
                p = self.random.normal(loc=0, scale=1, size=x.shape)
            else:
                x = np.hstack([tmp_alpha.reshape(tmp_alpha.shape[1]), tmp_alpha0])
                p = self.random.normal(loc=0, scale=1, size=x.shape)

            lambda_vec = np.exp(np.sum(tmp_alpha * x_train, axis=1) + tmp_alpha0)
            lambda_vec = lambda_vec.reshape([num, 1])
            if self.isGLMM == True:
                now_likelihood = py_vectorlize_quad(lambda_vec, y_train, tmp_gauss_s, self.max_dim).astype(float).reshape([num, 1])
            else:
                now_likelihood = py_vectorlize_non_quad(lambda_vec, y_train).astype(float).reshape([num, 1])
            now_hamiltonian = -np.sum(now_likelihood) / num + np.sum(p ** 2) / 2
            
            if self.isGLMM == True:
                hamiltonian_ε = np.array([self.hamiltonian_ε / np.mean(x_train), self.hamiltonian_ε, self.hamiltonian_ε])
                x, p = Leap_Flog_Cus(likelihood_GLMM_diff, x, p, ε=hamiltonian_ε, args=(self.max_dim,), max_iterate=self.hamiltonian_ite)
                tmp_alpha, tmp_alpha0, tmp_gauss_s = x[:-2].reshape(1, s), x[-2], x[-1]
            else:
                hamiltonian_ε = np.array([self.hamiltonian_ε / np.mean(x_train), self.hamiltonian_ε])
                x, p = Leap_Flog_Cus(likelihood_GLM_diff,  x, p, ε=hamiltonian_ε,                       max_iterate=self.hamiltonian_ite)
                tmp_alpha, tmp_alpha0              = x[:-1].reshape(1, s), x[-1]

            lambda_vec = np.exp(np.sum(tmp_alpha * x_train, axis=1) + tmp_alpha0)
            lambda_vec = lambda_vec.reshape([num, 1])
            if self.isGLMM == True:
                tmp_likelihood = py_vectorlize_quad(lambda_vec, y_train, tmp_gauss_s, self.max_dim).astype(float).reshape([num, 1])
            else:
                tmp_likelihood = py_vectorlize_non_quad(lambda_vec, y_train).astype(float).reshape([num, 1])
            tmp_hamiltonian = -np.sum(tmp_likelihood) / num + np.sum(p ** 2) / 2

            tmp_rand = self.random.random()
            tmp_rate = np.exp(now_hamiltonian - tmp_hamiltonian)
            if tmp_rand < tmp_rate:

                if now_ite % 1 == 0:
                    print(f"現在ite:{now_ite+1}  保存サンプリング数:{len(self.sampling_alpha0)}", flush=True)
                
                self.alpha, self.alpha0 = tmp_alpha, tmp_alpha0
                self.gauss_s            = tmp_gauss_s

                if now_ite > burnin:
                    if now_ite == (burnin + 1):
                        self.sampling_alpha   = np.array(self.alpha).reshape(  [1, s])
                        self.sampling_alpha0  = np.array(self.alpha0).reshape( [1, 1])
                        self.sampling_gauss_s = np.array(self.gauss_s).reshape([1, 1])
                    else:
                        self.sampling_alpha   = np.vstack((self.sampling_alpha,   self.alpha))
                        self.sampling_alpha0  = np.vstack((self.sampling_alpha0,  self.alpha0))
                        self.sampling_gauss_s = np.vstack((self.sampling_gauss_s, self.gauss_s))
                now_ite  = now_ite  + 1

        return True

    def predict(self, x_test, sample=100, step=1):
        self.alpha   = np.mean(self.sampling_alpha,   axis=0).reshape(self.alpha.shape)
        self.alpha0  = np.mean(self.sampling_alpha0,  axis=0).reshape(self.alpha0.shape)
        self.gauss_s = np.mean(self.sampling_gauss_s, axis=0).reshape(self.gauss_s.shape)

        if self.isGLMM == True:
            lambda_vec = np.exp(np.sum(self.alpha * x_test, axis=1) + self.alpha0)
            range_σ    = max(10 * self.gauss_s, 1)
            def vectorlize_quad(lambda_, y_test_indi):
                def mixture(r):
                    lambda_r  = lambda_ * np.exp(r)
                    poisson   = poisson_distribution(lambda_r, y_test_indi)
                    normal    = normal_distribution(r, 0, self.gauss_s)
                    return poisson * normal
                
                tmp = integrate.fixed_quad(mixture, -range_σ, range_σ, self.max_dim)[0]
                return y_test_indi, tmp

            y_test_mat = (lambda_vec - sample/2 * step).reshape([len(lambda_vec), 1])
            lambda_mat = (lambda_vec                  ).reshape([len(lambda_vec), 1])
            for ite in np.arange(-sample / 2 * step + step, sample / 2 * step, step):
                y_test_mat = np.hstack([y_test_mat, (lambda_vec + ite).reshape([len(lambda_vec), 1])])
                lambda_mat = np.hstack([lambda_mat, (lambda_vec      ).reshape([len(lambda_vec), 1])])

            pred_mat  = np.frompyfunc(vectorlize_quad, 2, 2)(lambda_mat, y_test_mat)
            pred_y    = pred_mat[0].astype(float)
            pred_prob = pred_mat[1].astype(float)
            
            return pred_y, pred_prob
        else:
            return np.exp(np.sum(self.alpha * x_test, axis=1) + self.alpha0)

In [ ]:
data_sets = agg_data_for_unit()

X = data_sets.drop(["date", "総売上点数", "総売上"], axis=1)
y = data_sets["総売上点数"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

model = PoissonRegression_On_Bayes(isGLMM=True)
model.sampling(X_train, y_train, iter_num=10000, burnin=0)

#data for 6400
#iterate  20  time  5m  5s
#iterate  30  time  7m 40s
#iterate  40  time 11m 12s
#iterate  50  time 13m 48s
#iterate  60  time 16m  9s
#iterate  70  time 18m 44s
#iterate  80  time 21m  6s
#iterate  90  time 23m 28s
#iterate 100  time 25m 49s

#predict
#iterate   1000  time      4h 19m 10s
#iterate  10000  time  1d 19h 11m 40s
#iterate 100000  time 17d 23h 56m 40s

#data for 3200
#iterate  10  time  1m 26s
#iterate  20  time  2m 43s
#iterate  30  time  4m 02s
#iterate  40  time  5m 28s
#iterate  50  time  6m 46s
#iterate  60  time  8m  3s
#iterate  70  time  9m 21s
#iterate  80  time 10m 39s
#iterate  90  time 12m  4s
#iterate 100  time 13m 21s

#predict
#iterate   1000  time      2h 12m 24s
#iterate  10000  time     22h 04m 04s
#iterate 100000  time  9d  4h 40m 44s

In [ ]:
plt.figure(figsize=(12, 9))
plt.subplots_adjust(wspace=0.15,hspace=0.4)

plt.subplot(3,2,1)
plt.plot(np.arange(0, model.sampling_alpha.shape[0], 1), model.sampling_alpha)
plt.title("alphaのサンプリング過程")
plt.xlabel("iterations")
plt.grid(True)

plt.subplot(3,2,2)
plt.hist(model.sampling_alpha[:, 0], bins=25)
plt.title("p(α|D)の確率密度関数")
plt.xlabel(f"N = {model.sampling_alpha.shape[0]}   σ = {0.01}")
plt.ylabel("頻度")
plt.grid(True)

plt.subplot(3,2,3)
plt.plot(np.arange(0, model.sampling_alpha0.shape[0], 1), model.sampling_alpha0)
plt.title("alpha0のサンプリング過程")
plt.xlabel("iterations")
plt.grid(True)

plt.subplot(3,2,4)
plt.hist(model.sampling_alpha0, bins=25)
plt.title("p(α0|D)の確率密度関数")
plt.xlabel(f"N = {model.sampling_alpha0.shape[0]}   σ = {0.01}")
plt.ylabel("頻度")
plt.grid(True)

plt.subplot(3,2,5)
plt.plot(np.arange(0, model.sampling_gauss_s.shape[0], 1), model.sampling_gauss_s)
plt.title("gauss_sのサンプリング過程")
plt.xlabel("iterations")
plt.grid(True)

plt.subplot(3,2,6)
plt.hist(model.sampling_gauss_s, bins=25)
plt.title("p(gs|D)の確率密度関数")
plt.xlabel(f"N = {model.sampling_gauss_s.shape[0]}   σ = {0.01}")
plt.ylabel("頻度")
plt.grid(True)

print("Statistics information")
print("          mean      median      std    ")
print("alpha     {:.5f}   {:.5f}     {:.5f}".format(np.mean(model.sampling_alpha  ), np.median(model.sampling_alpha  ), np.std(model.sampling_alpha  )))
print("alpha0    {:.5f}   {:.5f}     {:.5f}".format(np.mean(model.sampling_alpha0 ), np.median(model.sampling_alpha0 ), np.std(model.sampling_alpha0 )))
print("gauss_s   {:.5f}   {:.5f}     {:.5f}".format(np.mean(model.sampling_gauss_s), np.median(model.sampling_gauss_s), np.std(model.sampling_gauss_s)))
print("")

p_alpha   = np.percentile(model.sampling_alpha,   q=[0, 25, 50, 75, 100])
p_alpha0  = np.percentile(model.sampling_alpha0,  q=[0, 25, 50, 75, 100])
p_gauss_s = np.percentile(model.sampling_gauss_s, q=[0, 25, 50, 75, 100])
print("quartiles")
print("          2.5%    25%      50%      75%      97.5%")
print("alpha     {:.5f} {:.5f}  {:.5f}  {:.5f}  {:.5f}".format(p_alpha[0],   p_alpha[1],   p_alpha[2],   p_alpha[3],   p_alpha[4]))
print("alpha0    {:.5f} {:.5f}  {:.5f}  {:.5f}  {:.5f}".format(p_alpha0[0],  p_alpha0[1],  p_alpha0[2],  p_alpha0[3],  p_alpha0[4]))
print("gauss_s   {:.5f} {:.5f}  {:.5f}  {:.5f}  {:.5f}".format(p_gauss_s[0], p_gauss_s[1], p_gauss_s[2], p_gauss_s[3], p_gauss_s[4]))